In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

years = np.arange(2008, 2016)
names = ['Attività e Passività', 'Costi e Ricavi', 'Entrate', 'Spese']

In [2]:
columns = ['Denominazione', 'Dizione_Provincia', 'Dizione_Regione', 'Dizione_zona']

final = pd.read_csv('data//Anagrafe_comuni.csv', sep = ';', encoding = 'latin_1', low_memory = False).set_index('Id_Ente')

final['Data_Istituzione'] = pd.to_datetime(final['Data_Istituzione'], format='%Y-%m-%d')
final = final[final['Data_Istituzione'].dt.year < 2008]

final = final[final['Data_Cessazione'].isna()]

final = final[final['Codice_Tipologia_DLGS_118_2011'] == 'ELCOMU'][columns]

final = final.rename(columns={'Denominazione': 'Comune', 'Dizione_Provincia': 'Provincia', 'Dizione_Regione': 'Regione', 'Dizione_zona': 'Zona'})
final.index.names = ['BDAP']

final

,Comune,Provincia,Regione,Zona
BDAP,,,,
111142930451558302,COMUNE DI VERMIGLIO,TRENTO,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST
111142930456078402,COMUNE DI NARBOLIA,ORISTANO,SARDEGNA,ISOLE
111142930549694901,COMUNE DI RESANA,TREVISO,VENETO,NORD-EST
111242928159168602,COMUNE DI ALDINO,BOLZANO/BOZEN,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST
111342928035516202,COMUNE DI AUGUSTA,SIRACUSA,SICILIA,ISOLE
...,...,...,...,...
999842930521939101,COMUNE DI DOSOLO,MANTOVA,LOMBARDIA,NORD-OVEST
999842930529623202,COMUNE DI CESSOLE,ASTI,PIEMONTE,NORD-OVEST
999942930461543701,COMUNE DI BIENO,TRENTO,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST


In [3]:
final.to_pickle('comuni.pkl')

In [4]:
years = np.arange(2008, 2016)

# Zona

In [5]:
zona = pd.DataFrame(index = final.index)
for elem in final['Zona'].unique():
	zona[elem] = 0
for i, row in final.iterrows():
	zona.loc[i, row['Zona']] = 1

zona.to_pickle('zona.pkl')

# Popolazione

In [6]:
import os

def find_files(folder_path, search_string):
    matching_files = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if search_string in file:
                matching_files.append(os.path.join(root, file))

    return matching_files

def mean_age(data):
	 return np.mean(np.multiply(np.arange(0,101), data).sum())

In [7]:
comuni = pd.read_csv('data//Anagrafe_comuni.csv', sep = ';', encoding = 'latin_1', low_memory = False).set_index('Id_Ente')

In [8]:
popolazione = pd.DataFrame(index = final.index, columns = [f'pop_{year}' for year in years])

folder_path = 'popolazione//'

for idx in final.index:
	codice = str(comuni.loc[idx, 'Codice_ISTAT_Comune']).zfill(6)
	if codice == '099031':
		codice = '041060'
	if codice == '099030':
		codice = '041033'
	names = find_files(folder_path, codice)
	temp = pd.read_csv(names[0], sep = ';', encoding = 'latin_1', low_memory = False, skiprows=106, nrows=1).values[0][7:15]
	popolazione.loc[idx] = temp
	
popolazione = popolazione.apply(pd.to_numeric, errors='coerce')

In [9]:
popolazione.to_pickle('pop.pkl')

In [10]:
eta = pd.DataFrame(index = final.index, columns = [f'eta_{year}' for year in years])

folder_path = 'popolazione//'

for idx in final.index:
	codice = str(comuni.loc[idx, 'Codice_ISTAT_Comune']).zfill(6)
	if codice == '099031':
		codice = '041060'
	if codice == '099030':
		codice = '041033'
	names = find_files(folder_path, codice)
	temp = pd.read_csv(names[0], sep = ';', encoding = 'latin_1', low_memory = False, skiprows=4, nrows=102).values[1:,7:15].astype(int)
	temp2 = pd.read_csv(names[0], sep = ';', encoding = 'latin_1', low_memory = False, skiprows=106, nrows=1).values[0][7:15].astype(int)
	eta.loc[idx] = (temp* np.arange(temp.shape[0]).reshape(-1, 1)).sum(axis = 0)/temp2

eta = eta.apply(pd.to_numeric, errors='coerce')

In [11]:
eta.to_pickle('eta.pkl')

# Balance sheet items

In [ ]:
l1 = []
map1 = []
l2 = []
map2 = []

for name in names:
	filename = f'Dati 2008-2015//{name}.csv'
	bigdata = pd.read_csv(filename, sep = ';', encoding = 'latin_1', low_memory = False)
	
	if name == 'Costi e Ricavi':
		target = 'Valore Economico'
		slice1 = 'Descrizione CE I livello'
		slice2 = 'Descrizione CE II livello'
		
	elif name == 'Attività e Passività':
		target = 'Consistenza Finale Patrimonio'
		slice1 = 'Descrizione Voce I livello'
		slice2 = 'Descrizione Voce II livello'
		
	elif name == 'Entrate':
		target = 'Impegno totale'
		slice1 = 'Descrizione Titolo'
		slice2 = 'Descrizione Risorsa'
		
	else:
		target = 'Impegno totale'
		slice1 = 'Descrizione Titolo Spese'
		slice2 = 'Descrizione Intervento'
	
	print(f'############# DATASET:{name} #############')
	if name != 'Entrate':
		bigdata[target] = bigdata[target].str.replace(',', '.').astype(float)
	
	if name == 'Attività e Passività':
		bigdata.loc[(bigdata['Descrizione Voce I livello'] == 'RATEI E RISCONTI') & (bigdata['Tipologia Voce'] == 'Passivo') & (bigdata['Consistenza Finale Patrimonio'] < 0), 'Descrizione Voce I livello'] = 'RATEI E RISCONTI ATTIVO'
		bigdata.loc[(bigdata['Descrizione Voce I livello'] == 'RATEI E RISCONTI') & (bigdata['Tipologia Voce'] == 'Attivo') & (bigdata['Consistenza Finale Patrimonio'] < 0), 'Descrizione Voce I livello'] = 'RATEI E RISCONTI PASSIVO'
		bigdata.loc[(bigdata['Descrizione Voce I livello'] == 'RATEI E RISCONTI') & (bigdata['Tipologia Voce'] == 'Passivo') & (bigdata['Consistenza Finale Patrimonio'] >= 0), 'Descrizione Voce I livello'] = 'RATEI E RISCONTI PASSIVO'
		bigdata.loc[(bigdata['Descrizione Voce I livello'] == 'RATEI E RISCONTI') & (bigdata['Tipologia Voce'] == 'Attivo') & (bigdata['Consistenza Finale Patrimonio'] >= 0), 'Descrizione Voce I livello'] = 'RATEI E RISCONTI ATTIVO'
		bigdata['Consistenza Finale Patrimonio'] = bigdata['Consistenza Finale Patrimonio'].abs()

	if name == 'Entrate':
		bigdata['Riscossione in CC'] = bigdata['Riscossione in CC'].str.replace(',', '.').astype(float)
		bigdata['Riscossione in CR'] = bigdata['Riscossione in CR'].str.replace(',', '.').astype(float)
		bigdata['Impegno totale'] = bigdata['Riscossione in CC'] + bigdata['Riscossione in CR']

	if name == 'Entrate':
		unique_slice1 = sorted(bigdata[slice1].unique()[:-1])
		unique_slice2 = sorted(bigdata[slice2].unique())

	if name != 'Entrate':
		unique_slice1 = sorted(bigdata[slice1].unique())
		unique_slice2 = sorted(bigdata[slice2].unique())
		
	print(f'CAT I items:{len(unique_slice1)}')
	l1 += list(unique_slice2)
	map1 += [name for elem in unique_slice1]
	print(f'CAT II items:{len(unique_slice2)}')
	l2 += list(unique_slice2)
	map2 += [name for elem in unique_slice2]
	
	for year in years:

		temp = bigdata.loc[bigdata['Esercizio Finanziario'] == year].copy()
		
		temp_df_1 = temp.pivot_table(index='Codice BDAP Ente', columns = slice1, values = target, aggfunc='sum', fill_value=0)
		temp_df_2 = temp.pivot_table(index='Codice BDAP Ente', columns = slice2, values = target, aggfunc='sum', fill_value=0)
		
		temp_df_1 = temp_df_1.reindex(columns=unique_slice1, fill_value=0)
		temp_df_2 = temp_df_2.reindex(columns=unique_slice2, fill_value=0)
		
		temp_df_1 = temp_df_1.reindex(final.index, fill_value=0)
		temp_df_2 = temp_df_2.reindex(final.index, fill_value=0)
		
		print(f'{name} {year} CAT I\tShape: {temp_df_1.shape}\tNaN: {temp_df_1.isnull().sum().sum()} ')
		print(f'{name} {year} CAT II\tShape: {temp_df_2.shape}\tNaN: {temp_df_2.isnull().sum().sum()} ')
		temp_df_1.to_pickle(f'{name} {year} CAT I.pkl')
		temp_df_2.to_pickle(f'{name} {year} CAT II.pkl')
		
		del temp_df_1
		del temp_df_2
		gc.collect()
		
	del bigdata
	gc.collect()

# Aggregate Data

In [5]:
names = ['Attività e Passività', 'Costi e Ricavi', 'Entrate', 'Spese']
years = np.arange(2008, 2016)
dataframes = {}

for name in names:
	
    for year in years:
		
        for category in ["CAT I", "CAT II"]:
			
            file_path = f'data//{name} {year} {category}.pkl'
            df = pd.read_pickle(file_path)  
            dataframes[f'{name}_{year}_{category}'] = df

In [6]:
pop = pd.read_pickle('data//pop.pkl')

In [5]:
dataframes['Attività e Passività_2008_CAT I']

Descrizione Voce I livello,ATTIVO CIRCOLANTE,CONFERIMENTI,DEBITI,IMMOBILIZZAZIONI,PATRIMONIO NETTO,RATEI E RISCONTI ATTIVO,RATEI E RISCONTI PASSIVO
BDAP,,,,,,,
111142930451558302,0.0,0.0,1981179.0,0.0,0.0,0.0,0.0
111142930456078402,5043053.0,6727646.0,1211548.0,9955156.0,6725753.0,0.0,0.0
111142930549694901,6344642.0,6411162.0,2957843.0,18760554.0,15660588.0,5555.0,81158.0
111242928159168602,3489408.0,10189962.0,4619779.0,15272086.0,3967877.0,16124.0,0.0
111342928035516202,14507009.0,21823358.0,51532646.0,43385078.0,23238285.0,36585.0,0.0
...,...,...,...,...,...,...,...
999842930521939101,2381390.0,824135.0,3728711.0,8140679.0,5971143.0,1920.0,0.0
999842930529623202,812939.0,1597981.0,288007.0,2530855.0,1457806.0,0.0,0.0
999942930461543701,0.0,0.0,529038.0,0.0,0.0,0.0,0.0


In [7]:
final

,Comune,Provincia,Regione,Zona
BDAP,,,,
111142930451558302,COMUNE DI VERMIGLIO,TRENTO,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST
111142930456078402,COMUNE DI NARBOLIA,ORISTANO,SARDEGNA,ISOLE
111142930549694901,COMUNE DI RESANA,TREVISO,VENETO,NORD-EST
111242928159168602,COMUNE DI ALDINO,BOLZANO/BOZEN,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST
111342928035516202,COMUNE DI AUGUSTA,SIRACUSA,SICILIA,ISOLE
...,...,...,...,...
999842930521939101,COMUNE DI DOSOLO,MANTOVA,LOMBARDIA,NORD-OVEST
999842930529623202,COMUNE DI CESSOLE,ASTI,PIEMONTE,NORD-OVEST
999942930461543701,COMUNE DI BIENO,TRENTO,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST


In [21]:
features

['ADDIZIONALE COMUNALE SUL CONSUMO DI ENERGIA ELETTRICA',
 'ADDIZIONALE IRPEF',
 'ADDIZIONALE SUL CONSUMO DI ENERGIA ELETTRICA',
 'AGRICOLTURA',
 'AGRITURISMO',
 'ALIENAZIONE BENI IMMOBILI E DIRITTI REALI SU BENI IMMOBILI',
 'ALIENAZIONE BENI MOBILI E DIRITTI REALI SU BENI MOBILI',
 'ALIENAZIONE DI BENI PATRIMONIALI DIVERSI',
 'ALTRE ENTRATE DA CONTRIBUZIONE REGIONALE A SPECIFICA DESTINAZIONE PER FINANZIARE SPESE DI PERSONALE',
 'ALTRE IMPOSTE',
 'ALTRE TASSE',
 'ALTRI CONTRIBUTI',
 'ALTRI CONTRIBUTI E TRASFERIMENTI CORRENTI DA ALTRI ENTI DEL SETTORE PUBBLICO',
 'ALTRI CONTRIBUTI E TRASFERIMENTI CORRENTI DALLA REGIONE',
 'ALTRI CONTRIBUTI E TRASFERIMENTI CORRENTI DALLA REGIONE PER FUNZIONI DELEGATE',
 'ALTRI CONTRIBUTI E TRASFERIMENTI CORRENTI DALLO STATO',
 'ALTRI PROVENTI',
 "ALTRI PROVENTI DEI BENI DELL'ENTE",
 'ALTRI PROVENTI DEI BENI PROVINCIALI',
 'ALTRI PROVENTI DEI SERVIZI PUBBLICI',
 'ALTRI PROVENTI DIVERSI',
 'ALTRI PROVENTI DIVERSI DALLA FATTISPECIE PRECEDENTE',
 'ALTRI TRAS

In [22]:
feature_dfs = {}

def create_feature_dfs(features, category, dataset_name):
	for feature in features:
		df_feature = final.copy()
		
		for year in years:
			key = f'{dataset_name}_{year}_{category}'
			if key in dataframes:
				df_feature[year] = dataframes[key][feature]

		feature_dfs[f'{feature}_{category}'] = df_feature

		df_feature.to_csv(f'{feature}.csv')

cat = 'CAT I'

for dataset_name in names:
	sample_key = f'{dataset_name}_{years[0]}_{cat}'

	if sample_key in dataframes:
		features = list(dataframes[sample_key].columns)
		
		create_feature_dfs(features, cat, dataset_name)